In [163]:
import pandas as pd
import datetime as dt

from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import oandapyV20.endpoints.forexlabs as labs
# https://media.readthedocs.org/pdf/oanda-api-v20/latest/oanda-api-v20.pdf

In [164]:
client = op.API(access_token='7f736aabc877f3ea75bc844c79814d7c-998e41725285ea6d54b836e8b93fe4f6')

In [165]:
def hist(api, instrument, start_days, end_days, granularity):

    start_date = (dt.datetime.now()-dt.timedelta(days=start_days)).strftime('%Y-%m-%dT%H:%M:%SZ')
    end_date = (dt.datetime.now()-dt.timedelta(days=end_days, hours=2,minutes=4)).strftime('%Y-%m-%dT%H:%M:%SZ')

    params ={
                "from": start_date,
                "to": end_date,
                "granularity":granularity,
            }

    df_list = []
    for r in InstrumentsCandlesFactory(instrument=instrument,params=params):
        api.request(r)
        df = pd.DataFrame(r.response['candles'])
        if(df.empty==False):
            time = df['time']
            volume = pd.DataFrame(df['volume'].apply(pd.Series))
            df = pd.DataFrame(df['mid'].apply(pd.Series))
            df = pd.concat([df,time,volume], axis=1)
            df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%dT%H:%M:%S.000000000Z')
            #df.set_index('time',inplace=True)
            df_list.append(df)
    
    final = pd.concat(df_list)
    
    names = {
        'o': 'open',
        'c': 'close',
        'h': 'high',
        'l': 'low',
        0: 'vol',
        'time': 'time',
    }
    new_names = []
    for column_name in final.columns:
        new_names.append(names[column_name])
    final.columns = new_names
    
    return final

In [166]:
def cal(client, instrument, perdiod):

    
    
    params = {
        "instrument": instrument,
        "period": perdiod
    }
    
    # PERIOD VALUES
    #3600 - 1 hour
    #43200 - 12 hours
    #86400 - 1 day
    #604800 - 1 week
    #2592000 - 1 month
    #7776000 - 3 months
    #15552000 - 6 months
    #31536000 - 1 year
    # http://developer.oanda.com/rest-live/forex-labs/

    r = labs.Calendar(params=params)
    client.request(r)
    
    df = pd.DataFrame.from_dict(r.response, orient='columns')
    
    df['timestamp'] = pd.to_datetime(df['timestamp']*1000000000)
    df = df[['impact', 'timestamp']]
    df.columns = ['impact', 'time']

    return df

In [167]:
def merge(history, calendar):
    return pd.merge(history, calendar, left_on = 'time', right_on = 'time', how='outer')
    

In [168]:
calendar = cal(client, 'EUR_USD', 604800)

In [169]:
history = hist(client, 'EUR_USD', 7, 0, 'M5')

In [170]:
merged = merge(history, calendar)

In [172]:
# check the merge
merged[~merged['impact'].isna()].head()

,open,high,low,close,time,vol,impact
416,1.13604,1.13647,1.13564,1.13644,2019-01-22 10:00:00,279,3.0
417,1.13604,1.13647,1.13564,1.13644,2019-01-22 10:00:00,279,3.0
477,1.13561,1.13580,1.13544,1.13564,2019-01-22 15:00:00,165,2.0
1016,1.13440,1.13450,1.13414,1.13423,2019-01-24 11:45:00,86,3.0
1017,1.13440,1.13450,1.13414,1.13423,2019-01-24 11:45:00,86,3.0


In [ ]:
# TODO

# calendar - flag dangerous periods
# oanda lab - indicators
# ta - indicators

# divide into smaller dataframes ???

# for each learning period train model and check acc
# so if learning period is one week repeat process every day

# pick the most stable and profitable model - not appliable kurwa xD
# how to measure success???
# if not profitable, change parameters - which ones???
